# UCN LD2 Calculations: Helical Groove

K. Augusto

June 1/20

In [2]:
#!/usr/bin/python3

from math import *

from mpl_toolkits import mplot3d

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy import *

Parameters:

Pressure = 20 psi

Temperature in = T$_{in}$ = 23.4 K

Temperature of the wall = T$_w$ = 20.7 K 

Mass flow rate = $\dot{m}$ = 0.004 kg/s

Thermal conductivity = $\mu$ = 3.5$\times10^{-5}$ Pa s

Length of HEX = L = 10 in = 0.254 m 

Density of D2 = $\rho$ = 163 kg/m$^3$ 

Specific heat = C$_p$ = 6565.4 J/(kg K)

Number of grooves = N = 1

Diameter of tube before groove(s) cut = D = 4.76 in = 0.1209 m 

Width of groove(s) = w' = 0.01 m 

Width between groove(s) = u' = 0.01 m 

Depth of groove(s) = depth = 0.01 m 


In [19]:
p_psi=20. # PSI
p=p_psi*6894.76 # Pa
kt=0.104 # W/(m*K) a
T=Tin=23.4 # (K) inlet temp
Tw=20.7 # (K) temperature of cold wall
mdot=0.004 # kg/s
mu=3.5e-5 # Pa*s
L=10*0.0254 #m length of tube
rho=163.0 # kg/m^3
Cp=6565.4 # J/(kg*K)
Ngrooves=1 # number of grooves where it was found that the optimal groove is 1
D=4.76*0.0254 #m diameter of tube, 0.015949 from optimizing dp in backwards-hex-turbulent-tube.py
wprime= 0.01 #m width of groove
uprime= 0.01 # m width between grooves
depth=0.01 # m depth of groove

sinalpha=(Ngrooves*(wprime + uprime))/(pi*D) #pitch angle

alpha=arcsin(sinalpha)

print('The pitch angle is %f' %sinalpha)

The pitch angle is 0.052655


In [5]:
Lprime=L/sinalpha #m length of wound groove

print('The length of the groove is %f m.' %Lprime)

turns=Lprime/(pi*D)

print('Coiling around a Cu rod of diameter %f m would require %f turns'%(D,turns))

The length of the groove is 4.823855 m.
Coiling around a Cu rod of diameter 0.120904 m would require 12.700000 turns


The flow area of the helical fin is found from:

\begin{equation}
A = Nw'depth
\end{equation}

The flow perimeter of the helical fin is found from:

\begin{equation}
P = 2N(depth + w')
\end{equation}

In [6]:
w=wprime*tan(alpha) # m

ahelix=Ngrooves*wprime*depth # m^2 area of one helical groove/fin thing

print('The area of the helical fins is %f m^2.'%ahelix)

phelix=Ngrooves*(2*depth+2*wprime) #m

print('The perimeter of the helical grooves is %f m.' %phelix)

The area of the helical fins is 0.000100 m^2.
The perimeter of the helical grooves is 0.040000 m.


The hydraluic diameter is found from:

\begin{equation}
D_h = \frac{4A}{P}
\end{equation}

The Mass flux per unit area is found from:

\begin{equation}
G = \frac{\dot{m}}{A}
\end{equation}

In [20]:
Dh=4*ahelix/phelix #m

print('Hydraulic diameter %f m'%Dh)
print()

G=mdot/ahelix # (kg/(m^2*s)) mass flow rate per unit area

print('Mass flux per unit area is %f kg/(m^2*s)'%G)

Hydraulic diameter 0.010000 m

Mass flux per unit area is 40.000000 kg/(m^2*s)


The Reynolds Number is found from:

\begin{equation}
Re = \frac{D_hG}{\mu}
\end{equation}

In [8]:
Re=Dh*G/mu # should be dimensionless
print('The Reynolds number is %f'%Re)

The Reynolds number is 11428.571429


The friction factor is found from:

\begin{equation}
f = 0.316Re^{(-1/4)}
\end{equation}

In [9]:
fRe=24.00*4

if Re < 2300 :
    f=fRe/Re
    #f=64/Re #assuming cicular tube
    print('The laminar friction factor is %f.' %f)
elif 3500 > Re > 2300 :
    f=1.2036*Re**(-0.416) #from vijayan
    print('The friction factor is in between laminar and turbulent')
elif Re > 3500 :
    f=0.316*Re**(-0.25)
    print('The turbulent friction factor is %f.' %f)

The turbulent friction factor is 0.030563.


The Colburn factor is found from:

\begin{equation}
jH = 0.023Re^{-1/5}B_1
\end{equation}

In [10]:
B1=1.174*((3.7e-5)/(3.68e-5))**(0.14) #viscosity taken from cams sheets
print('This is B1 %f.' %B1)

if Re < 3500 :
    print('It is laminar or in between')
elif Re > 3500 :
    jh=0.023*Re**(-0.2)*B1
    print('The Colburn factor for the turbulent flow is %f.' %jh)

This is B1 1.174891.
The Colburn factor for the turbulent flow is 0.004170.


The Prandelt Number is found from:

\begin{equation}
Pr = \frac{\mu C_p}{k_t}
\end{equation}

In [14]:
Pr=(mu*Cp)/(kt) # yes still dimensionless

print('The Prandtl Number is %f.'%Pr)

The Prandtl Number is 2.209510.


The Nusselt Number is found from:

\begin{equation}
Nu = jHRePr^{1/3}
\end{equation}

In [15]:
if Re < 3500 :
    Nu=4.8608
    print('Nu=4.8608 because the flow is laminar')
elif Re > 3500 :
    Nuturb=jh*Re*Pr**(1./3.)
    print('This is the turbulent Nusselt Number %f.' %Nuturb)
    
print()

This is the turbulent Nusselt Number 62.070370.



The heat transfer coefficient is found from:

\begin{equation}
hc = \frac{Nuk_t}{D_h}
\end{equation}

In [16]:
if Re < 3500 :
    hc=Nu*kt/Dh # Barron eq'n 6.15
    print('The heat transfer coefficient for laminar flow is %f W/(m^2*K)'%hc)
elif Re > 3500 :
    hc=Nuturb*kt/Dh # Barron eq'n 6.17 makes it incredibly tiny compared to eq'n 6.15 maybe should be using eq'n 6.40 ??
    print('The heat transfer coefficient for turbulent flow is %f W/(m^2*K)'%hc)

The heat transfer coefficient for turbulent flow is 645.531850 W/(m^2*K)


The pressure drop is found from:

\begin{equation}
\Delta P = \frac{fLG^2}{2D_h\rho} \\
\end{equation}

In [17]:
dp=(f*L*G**2)/(Dh*2*rho) # (Pa) pressure drop

print('The pressure drop is %f Pa'%dp)

The pressure drop is 3.810002 Pa
